# Requires
Permit Status:
- Approved
- Refused
- Withdrawn
- Cancelled

Permit Description \
Permit Applicant Name \
Permit Number

In [6]:
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
import time

In [7]:
url = 'https://www.shapeyourcity.ca/development'
PATH = "C:\Program Files (x86)\chromedriver-win64\chromedriver.exe"

In [17]:
driver = webdriver.Chrome(service = Service(PATH))
driver.get(url)
iframes = driver.find_elements(By.TAG_NAME, "iframe")
url_list = []
for num in range(2):
    for iframe in iframes:
        driver.switch_to.frame(iframe)
        link = WebDriverWait(driver, 20).until(EC.visibility_of_all_elements_located((By.CSS_SELECTOR, '.chakra-link.ehq-projectCoverImg.css-1eh7kaa')))
        for item in link:
            url_list.append(item.get_attribute('href'))
        driver.switch_to.default_content()
    driver.switch_to.frame(iframes[0])
    click = driver.find_element(By.CSS_SELECTOR, ".chakra-button.ehq-paginationButton.ehq-paginationNextButton.css-i1louw")
    click.click()
    driver.switch_to.default_content()
    time.sleep(5)
url_list

['https://www.shapeyourcity.ca/436-e-hastings-st',
 'https://www.shapeyourcity.ca/985-nicola-st',
 'https://www.shapeyourcity.ca/3581-w-30-ave-2',
 'https://www.shapeyourcity.ca/848-seymour-st-2',
 'https://www.shapeyourcity.ca/2775-e-hastings-st',
 'https://www.shapeyourcity.ca/1338-matthews-ave',
 'https://www.shapeyourcity.ca/1090-w-pender-st',
 'https://www.shapeyourcity.ca/385-lakewood-dr',
 'https://www.shapeyourcity.ca/3048-3080-w-broadway',
 'https://www.shapeyourcity.ca/691-w-28-ave-2',
 'https://www.shapeyourcity.ca/2518-2540-grandview-hwy-s-2',
 'https://www.shapeyourcity.ca/1208-w-40-ave',
 'https://www.shapeyourcity.ca/314-328-w-hastings-st-2',
 'https://www.shapeyourcity.ca/7190-oak-st',
 'https://www.shapeyourcity.ca/2607-ward-st',
 'https://www.shapeyourcity.ca/6518-main-st',
 'https://www.shapeyourcity.ca/233-e-11-ave',
 'https://www.shapeyourcity.ca/2629-w-49-ave']

In [18]:
len(url_list)

18

In [ ]:
urls = item.get_attribute('href') for item in link